In [1]:
import requests
import tensorflow as tf
import numpy as np
import json
import matplotlib.pyplot as plt
import base64

In [2]:
def prepare_json(text):
    # Create an Example proto from your feature dict.
    feature_spec = {
        # "review" merupakan feature key dari data
        "review": tf.train.Feature(bytes_list=tf.train.BytesList(value=[bytes(text, "utf-8")]))
    }
    example = tf.train.Example(
      features=tf.train.Features(feature=feature_spec)).SerializeToString()
    result = [{'examples': {'b64': base64.b64encode(example).decode()}}]
    
    return json.dumps(
        {
            "signature_name":"serving_default",
            "instances": result
        }
    )

In [31]:
test_data = [
    "The movie was fantastic! I really enjoyed it.",
    "I didn't like the movie at all. It was boring."
]

endpoint = "http://localhost:8080/v1/models/roastland-pipeline:predict"

result = {}

for text in test_data:
    response = requests.post(endpoint, data=prepare_json(text))
    if (response.json()["predictions"][0][0] >= 0.5):
        result[text] = "Positive"
    else:
        result[text] = "Negative"
    
print(result)

{'The movie was fantastic! I really enjoyed it.': 'Positive', "I didn't like the movie at all. It was boring.": 'Negative'}
